In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1606972783248_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql.types import ArrayType, IntegerType, DoubleType, StringType, StructField, StructType
import pyspark.sql.functions as F
from pyspark.ml import Pipeline
from pyspark.ml.feature import Imputer, StandardScaler, VectorAssembler, BucketedRandomProjectionLSH, OneHotEncoder

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
all_cols = [
    "artist_familiarity", "artist_hotttnesss",
    "release_7digitalid", "song_id", "song_hotttnesss",
    "track_7digitalid", "danceability", "energy", "key",
    "loudness", "mode", "tempo", "time_signature", "duration",
    "year"
]

cont_cols = [
#     "artist_familiarity",
    "artist_hotttnesss",
#     "song_hotttnesss",
#     "danceability",
#     "energy",
    "loudness",
    "tempo",
    "duration",
    "year"
]

cat_cols = [
    "key", "mode", "time_signature"
]

schema = (StructType()
          .add("artist_familiarity", DoubleType(), True)
          .add("artist_hotttnesss", DoubleType(), True)
          .add("release_7digitalid", StringType(), True)
          .add("song_id", StringType(), True)
          .add("song_hotttnesss", DoubleType(), True)
          .add("track_7digitalid", StringType(), True)
          .add("danceability", DoubleType(), True)
          .add("energy", DoubleType(), True)
          .add("key", IntegerType(), True)
          .add("loudness", DoubleType(), True)
          .add("mode", IntegerType(), True)
          .add("tempo", DoubleType(), True)
          .add("time_signature", IntegerType(), True)
          .add("duration", DoubleType(), True)
          .add("year", DoubleType(), True))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
songs_df = (spark.read.format("csv")
            .option("header", False)
            .option("nanValue", "nan")
            .schema(schema)
            .load("s3://millionsongs10605-single/*.csv")
            .withColumn("song_id", F.expr("substring(song_id, 3, length(song_id)-3)")))
songs_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-------------------+------------------+------------------+------------------+----------------+------------+------+---+--------+----+-------+--------------+---------+------+
|artist_familiarity|  artist_hotttnesss|release_7digitalid|           song_id|   song_hotttnesss|track_7digitalid|danceability|energy|key|loudness|mode|  tempo|time_signature| duration|  year|
+------------------+-------------------+------------------+------------------+------------------+----------------+------------+------+---+--------+----+-------+--------------+---------+------+
|0.6406103635550155|0.45291165357852886|            769519|SOYHVGK12AB018A85C|               NaN|         8522639|         0.0|   0.0|  7|  -11.72|   1|145.521|             1| 146.9122|1969.0|
|0.5317360732062095|0.40913307872205695|            312013|SOXLFQD12A8C141927|               NaN|         3521635|         0.0|   0.0|  7|  -7.446|   1|  84.53|             4|211.59138|1997.0|
|0.7975710791520922|  0.56273293357

In [17]:
imputer = Imputer(
    inputCols=["artist_familiarity", "artist_hotttnesss"],
    outputCols=["artist_familiarity", "artist_hotttnesss"],
    strategy="mean"
)

cont_assembler = VectorAssembler(
    inputCols=cont_cols,
    outputCol="raw_cont_features"
)

scaler = StandardScaler(
    inputCol="raw_cont_features",
    outputCol="cont_features",
    withStd=True, withMean=True)

ohe_cat_cols = [col + "_ohe" for col in cat_cols]
ohes = [
    OneHotEncoder(
        inputCol=col,
        outputCol=ohe_col)
    for col, ohe_col in zip(cat_cols, ohe_cat_cols)]

all_assembler = VectorAssembler(
    inputCols=["cont_features"] + ohe_cat_cols,
    outputCol="all_features"
)

brp = BucketedRandomProjectionLSH(
    inputCol="all_features",
    outputCol="hashes",
    bucketLength=1.0,
    numHashTables=3,
    seed=420)

pipeline = Pipeline(stages=[
    imputer, cont_assembler, scaler
] + ohes + [all_assembler])

prep_model = pipeline.fit(songs_df)

prep_songs_df = prep_model.transform(songs_df)
brp_model = brp.fit(prep_songs_df)

out_songs_df = brp_model.transform(prep_songs_df).cache()
out_songs_df.select("song_id", "all_features", "hashes").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+--------------------+
|           song_id|        all_features|              hashes|
+------------------+--------------------+--------------------+
|SOYHVGK12AB018A85C|(24,[0,1,2,3,4,12...|[[-1.0], [0.0], [...|
|SOXLFQD12A8C141927|(24,[0,1,2,3,4,12...|[[-1.0], [0.0], [...|
|SOCSITI12AC3DF7F35|(24,[0,1,2,3,4,6,...|[[-1.0], [0.0], [...|
|SOKNJZO12A8C139449|(24,[0,1,2,3,4,8,...|[[-1.0], [1.0], [...|
|SOFFLLP12AB018ED52|(24,[0,1,2,3,4,14...|[[-1.0], [-1.0], ...|
+------------------+--------------------+--------------------+
only showing top 5 rows

In [18]:
key = out_songs_df.select("all_features").first().all_features
(brp_model
 .approxNearestNeighbors(out_songs_df, key, 10)
 .select("song_id", "distCol")
 .show(10, False))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-------------------+
|song_id           |distCol            |
+------------------+-------------------+
|SOYHVGK12AB018A85C|0.0                |
|SOHMOEL12A8C13F854|0.17679950166389155|
|SOKEWOC12A8C143160|0.18657421764537885|
|SOXQEXQ12AB018246E|0.21121106093768907|
|SOEANWR12A8C1445F7|0.21828759807230144|
|SOAPBED12AB0187A5B|0.22982641440853654|
|SOKUATM12A6311082E|0.2772365187843066 |
|SOACMWD12A6701E9C4|0.2793686172450861 |
|SOMKOPS12AB018799B|0.2817550020548286 |
|SOONCQA12AC9097A0D|0.3218561021924457 |
+------------------+-------------------+

In [19]:
test_visible_df = (sc.textFile("s3://millionsongs10605/EvalDataYear1MSDWebsite/year1_test_triplets_visible.txt")
        .map(lambda l: l.split("\t"))
        .toDF(["user_id", "song_id", "count"]))

test_hidden_df = (sc.textFile("s3://millionsongs10605/EvalDataYear1MSDWebsite/year1_test_triplets_hidden.txt")
        .map(lambda l: l.split("\t"))
        .toDF(["user_id", "song_id", "count"]))

test_visible_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+-----+
|             user_id|           song_id|count|
+--------------------+------------------+-----+
|00007a02388c208ea...|SOAITVD12A6D4F824B|    3|
|00007a02388c208ea...|SONZGLW12A6D4FBBC1|    1|
|00007a02388c208ea...|SOXNWYP12A6D4FBDC4|    1|
|00007a02388c208ea...|SOSPTOE12A6D4F938C|    1|
|00007a02388c208ea...|SOXMOQF12A6701FB8B|    1|
+--------------------+------------------+-----+
only showing top 5 rows

In [20]:
df_queries = (test_visible_df
              .join(out_songs_df, on="song_id", how="inner")
              .select("user_id", "song_id", "count", "all_features", "hashes"))
print(df_queries.count())
df_queries.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1357197
+--------------------+------------------+-----+--------------------+--------------------+
|             user_id|           song_id|count|        all_features|              hashes|
+--------------------+------------------+-----+--------------------+--------------------+
|1efc7ed552aa17146...|SOAAAFI12A6D4F9C66|    1|(24,[0,1,2,3,4,12...|[[-1.0], [0.0], [...|
|d075a43b518c724d8...|SOAAAFI12A6D4F9C66|    1|(24,[0,1,2,3,4,12...|[[-1.0], [0.0], [...|
|01a85f046df67f633...|SOAAAGQ12A8C1420C8|    5|(24,[0,1,2,3,4,14...|[[0.0], [1.0], [1...|
|05b340468f70fee0b...|SOAAAGQ12A8C1420C8|    1|(24,[0,1,2,3,4,14...|[[0.0], [1.0], [1...|
|0b91b7ba26eea486b...|SOAAAGQ12A8C1420C8|    1|(24,[0,1,2,3,4,14...|[[0.0], [1.0], [1...|
+--------------------+------------------+-----+--------------------+--------------------+
only showing top 5 rows

In [21]:
from pyspark.sql import Window

w = Window.partitionBy("user_id")
df_top_queries = (df_queries
 .withColumn("maxCount", F.max("count").over(w))
 .where(F.col("count") == F.col("maxCount"))
 .drop("maxCount"))

df_top_queries.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+-----+--------------------+--------------------+
|             user_id|           song_id|count|        all_features|              hashes|
+--------------------+------------------+-----+--------------------+--------------------+
|00014a76ed063e1a7...|SOTCMDJ12A6D4F8528|    4|(24,[0,1,2,3,4,6,...|[[0.0], [1.0], [0...|
|0018e70be70ebbdf1...|SONHQAZ12A8C1417B8|    2|(24,[0,1,2,3,4,14...|[[0.0], [1.0], [0...|
|0029bec66fe7ce2a3...|SOFXDVI12AB0187514|    7|(24,[0,1,2,3,4,6,...|[[-1.0], [1.0], [...|
|00526ad22352144fd...|SOCRYUK12AC95F0772|    5|(24,[0,1,2,3,4,8,...|[[-1.0], [1.0], [...|
|00526ad22352144fd...|SOQHOUQ12A6D4F8DA9|    5|(24,[0,1,2,3,4,13...|[[-1.0], [0.0], [...|
+--------------------+------------------+-----+--------------------+--------------------+
only showing top 5 rows

In [22]:
print(out_songs_df.count())
print(df_top_queries.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1000000
218997

In [23]:
sims = brp_model.approxSimilarityJoin(out_songs_df, df_top_queries, 0.5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
preds_df = (sims
 .select("datasetA.song_id", "datasetB.user_id", "distCol")
 .orderBy("distCol")
 .groupBy("user_id")
 .agg(F.collect_list("song_id").alias("preds"))
 .withColumn("preds", F.slice("preds", 2, 10)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
ground_truths = (test_hidden_df
                .orderBy("count", ascending=False)
                .groupBy("user_id")
                .agg(F.collect_list("song_id").alias("ground_truths")))

ground_truths.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+
|             user_id|       ground_truths|
+--------------------+--------------------+
|00014a76ed063e1a7...|[SONNHSO12A58A7D2...|
|0018e70be70ebbdf1...|[SOVPAJA12A58A77B...|
|0029bec66fe7ce2a3...|[SOPVCJR12AB0186A...|
+--------------------+--------------------+
only showing top 3 rows

In [ ]:
from pyspark.mllib.evaluation import RankingMetrics

compare = (ground_truths
           .join(preds_df, on="user_id")
           .select("preds", "ground_truths")
           .rdd
           .map(lambda r: (r.preds, r.ground_truths)))

metrics = RankingMetrics(compare)
prec = metrics.precisionAt(10)
print(prec)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.0003047021308903178

In [ ]:
prec